In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
import re
import pickle
from function_library import *
from function_library2 import *

## Find out all the people related in Edison project

In [ ]:
# load people list from pickle
# maybe should save body and title into pickle as well?
l_from = fromPickle("from_list")
l_to   = fromPickle("to_list")
l_cc   = fromPickle("cc_list")

In [ ]:
# load df with body
df = pd.read_csv('output4.csv')
df['Sent'] = pd.to_datetime(df['Sent'])
df = df[df['Sent'] > datetime( 2012, 1, 1, 0, 0, 0)]
body = df['Body'].values
subject = df['Subject'].values

In [ ]:
# if edison appear in the subject or the body, save the index
# GE: you can probably get faster execution using the pd.apply function
edison = 'edison'
idx_list = []
for idx in range(len(body)):
    if edison in body[idx].lower() or edison in subject[idx].lower():
        idx_list.append(idx)

In [ ]:
# find all the senders using the index list 
from_list = standardize_triplet(np.array(l_from)[idx_list].tolist())
name_list = set(from_list)

# find all the recipients using the index list 
to_list = standardize_triplet(np.array(l_to)[idx_list].tolist())
for names in to_list:
    # GE: I do not follow the logic
    name_list |= set(names)
cc_list = standardize_triplet(np.array(l_cc)[idx_list].tolist())
for names in cc_list:
    name_list |= set(names)

# convert to list and sort, in order to save to a txt
name_list = list(name_list)
name_list.sort()
#name_list[0:5]

In [ ]:
# save to a text file
# GE: person[i] is the triplet (first name, last name, email)
# GE: person[0] is ('','',''). Is this always true? Or does it depend on the data? 
# GE:    if not always true, the code below will not always work.
with open('edison_people_list.txt', 'w') as filehandle:
    for person in name_list[1:]:
        filehandle.write('{} {}: {}\n'.format(person[0], person[1], person[2]))

## Pick out all the related emails

In [ ]:
# pick out all the related emails and save into another csv
# GE: make sure you explain to our team what criteria you used to extract Edison-related emails

df_edison = df.iloc[idx_list]

# GE: Why are there 'Unnamed: 0' in df_edison? I thought we had cleaned up the emails
# GE: to make sure these columns do not appear, you must use pd.read_csv and/or df.to_csv
# GE: with an additional argument: 
del df_edison['Unnamed: 0']
del df_edison['Unnamed: 0.1']

df_edison.to_csv('edison_emails.csv')
#df_edison.head(50)

In [ ]:
# number of emails related 
len(idx_list)

## Find abnormal action by looking at sending time
* **All emails are used**  
* **The sending time is divided into 4 ranges. The idea is to focus on the emails sent outside regular work hours**  

In [ ]:
# identify the sending time in 4 time ranges
l_hour = fromPickle("hour_list")
l_hour_partition = []
for hour in l_hour:
    if hour>=4 and hour<=7:
        l_hour_partition.append(1)
    elif hour>=8 and hour<=16:
        l_hour_partition.append(2)
    elif hour>=17 and hour<=20:
        l_hour_partition.append(3)
    else:
        l_hour_partition.append(4)
toPickle(l_hour_partition, 'hour_partition')       

In [ ]:
l_year = fromPickle("year_list")
l_week = fromPickle("week_list")
l_month = fromPickle("month_list")
l_dayofyear = fromPickle("dayofyear_list")

In [ ]:
# create a matrix, #year by #day by #time_range+1
# GE: what do the dimensionsof hour_range_by_day represent?
hour_range_by_day = np.zeros((6,366,5))
for i in range(len(l_hour_partition)):
    ix = l_year[i]-2012
    iy = l_dayofyear[i]-1
    iz = l_hour_partition[i]-1
    # GE: explain the next two lines.
    hour_range_by_day[ix,iy,iz] += 1    
    hour_range_by_day[ix,iy,4] += 1

In [ ]:
time_range_labels = ['before work (4am-8am)','work time (8am-5pm)', 'after work (5pm-9pm)', 'night time (9pm-4am)']

plt.figure(figsize = (15,3))
for i in range(4):
    #GE: why are the first two indices of hour_range_by_dan?
    temp = np.reshape(hour_range_by_day[:,:,i], -1)
    plt.plot(temp, label = time_range_labels[i])
plt.legend()
plt.title('# of emails in 4 time partitions')
plt.show()

hour_partition_sum = np.reshape(hour_range_by_day[:,:,4], -1)
plt.figure(figsize = (15,3))
for i in range(4):
    temp = np.reshape(hour_range_by_day[:,:,i], -1)
    plt.plot(temp/hour_partition_sum, label = time_range_labels[i])
plt.legend()
plt.title('fraction of emails in 4 time partitions')
plt.show()


**Using day as the base unit may be too dense. 12000 emails in 2000 days, six per day in average.**  
**So I try to use a weekly scale below**

In [ ]:
# create a matrix, #year by #week by #time_range+1
hour_range_by_week = np.zeros((6,52,5))
for i in range(len(l_hour_partition)):
    ix = l_year[i]-2012
    iy = l_week[i]-1
    iz = l_hour_partition[i]-1
    if iy == 52:
        continue
    hour_range_by_week[ix,iy,iz] += 1   # GE: iz = 0,1,2,3
    hour_range_by_week[ix,iy,4] += 1 # GE: what is this for?


In [ ]:
plt.figure(figsize = (15,3))
for i in range(4):
    temp = np.reshape(hour_range_by_week[:,:,i], -1)
    plt.plot(temp, label = time_range_labels[i])
plt.legend()
plt.title('# of emails in 4 time partitions')
plt.show()

hour_partition_sum = np.reshape(hour_range_by_week[:,:,4], -1)
plt.figure(figsize = (15,3))
for i in range(4):
    temp = np.reshape(hour_range_by_week[:,:,i], -1)
    plt.plot(temp/hour_partition_sum, label = time_range_labels[i])
plt.legend()
plt.title('fraction of emails in 4 time partitions')
plt1 = plt
plt.show()


## Only apply on Edison emails

In [ ]:
df = pd.read_csv('edison_emails.csv')
df['Sent'] = pd.to_datetime(df['Sent'])
del df['Unnamed: 0']
df['year']    = df['Sent'].dt.year
df['month']   = df['Sent'].dt.month
df['week']    = df['Sent'].dt.week
df['weekday'] = df['Sent'].dt.weekday
df['day'] = df['Sent'].dt.day
df['dayofyear'] = df['Sent'].dt.dayofyear
df['hour']    = df['Sent'].dt.hour
# df.head()

In [ ]:
# identify the sending time in 4 time ranges

# GE: this is 100% identical to the cells above for the entire set of emails?

l_hour = df['hour'].tolist()
l_hour_partition = []
for hour in l_hour:
    if hour>=4 and hour<=7:
        l_hour_partition.append(1)
    elif hour>=8 and hour<=16:
        l_hour_partition.append(2)
    elif hour>=17 and hour<=20:
        l_hour_partition.append(3)
    else:
        l_hour_partition.append(4)
toPickle(l_hour_partition, 'hour_partition')       

In [ ]:
l_year = df['year'].tolist()
l_week = df['week'].tolist()
l_month = df['month'].tolist()
l_dayofyear = df['dayofyear'].tolist()

In [ ]:
# create a matrix, #year by #week by #time_range+1
hour_range_by_week = np.zeros((6,52,5))
for i in range(len(l_hour_partition)):
    ix = l_year[i]-2012
    iy = l_week[i]-1
    iz = l_hour_partition[i]-1
    if iy == 52:
        continue
    hour_range_by_week[ix,iy,iz] += 1    
    hour_range_by_week[ix,iy,4] += 1


In [ ]:
plt.figure(figsize = (15,3))
for i in range(4):
    temp = np.reshape(hour_range_by_week[:,:,i], -1)
    plt.plot(temp, label = time_range_labels[i])
plt.legend()
plt.title('# of emails in 4 time partitions')
plt.show()

hour_partition_sum = np.reshape(hour_range_by_week[:,:,4], -1)
plt.figure(figsize = (15,3))
for i in range(4):
    temp = np.reshape(hour_range_by_week[:,:,i], -1)
    plt.plot(temp/hour_partition_sum, label = time_range_labels[i])
plt.legend()
plt.title('fraction of emails in 4 time partitions')
plt.show()


## Exploring style analysis using POS

In [ ]:
df = pd.read_csv('new_clean_output.csv')
df['Sent'] = pd.to_datetime(df['Sent'])
df = df[df['Sent'] > datetime( 2012, 1, 1, 0, 0, 0)]
df.head()

In [ ]:
# GE: this is per email? Do you also analyze the subjects, or not? 
df_pos = pd.read_csv('pos_counts.csv')  # GE: for all emails
del df_pos['Unnamed: 0']
df_pos_edison = df_pos.iloc[idx_list]   # GE: only for Edison emails
unique_pos_tags = df_pos_edison.columns # GE: what do you mean by unique_pos_tags?
df_pos_edison.head()

In [ ]:
# convert to numpy array
mat_pos_edison = df_pos_edison.values

In [ ]:
nb_top = 50
plt.figure(figsize = (18,3))
for i in range(len(idx_list)):
    row = mat_pos_edison[i,:]
    if row.sum() > 5000:
        continue
    plt.bar(np.arange(0,len(row),1),row, alpha = 0.5, label = 'email {}'.format(i) )
    if i == nb_top:
        break
plt.xticks(np.arange(0,len(row),1), unique_pos_tags)
#plt.legend()
plt.title('Showing the POS distribution of first {} emails in Edison project'.format(nb_top))
plt.show()

# GE: YOu could create a table of what the different POS labels mean

In [ ]:
# some emails are extremely long. I put them in a list for further use
long_email_idx_list = []
for i in range(len(idx_list)):
    row = mat_pos_edison[i,:]
    if row.sum() > 5000:  #GE: emails > 5000 characters long
        long_email_idx_list.append(i)
long_email_idx_list

In [ ]:
# get propotion of pos in each email
m,n = mat_pos_edison.shape
mat_pos_edison_normed = np.zeros_like(mat_pos_edison)
for i in range(m):
    if mat_pos_edison[i,:].sum() != 0:
        mat_pos_edison_normed[i,:] = mat_pos_edison[i,:]/mat_pos_edison[i,:].sum()

In [ ]:
df_pos_edison_normed = pd.DataFrame(mat_pos_edison_normed, columns=unique_pos_tags)
df_pos_edison_normed.head()

In [ ]:
nb_top = 10
plt.figure(figsize = (18,3))
for i in range(len(idx_list)):
    row = mat_pos_edison_normed[i,:]
    if i in long_email_idx_list:
        continue
    plt.bar(np.arange(0,len(row),1),row, alpha = 0.5, label = 'email {}'.format(i) )
    if i == nb_top:
        break
plt.xticks(np.arange(0,len(row),1), unique_pos_tags)
plt.legend()
plt.title('Showing the POS proportion of first {} emails in Edison project'.format(nb_top))
plt.show()

## Find the most different email by comparing the distance of POS vector for each email and the average vector

In [ ]:
proportion_dists = []
proportion_mean = mat_pos_edison_normed.mean(axis=0)
for i in range(len(idx_list)):
#     if i in long_email_idx_list:
#         proportion_dists.append(-1)
#         continue
    row = mat_pos_edison_normed[i,:]
    proportion_dists.append(np.linalg.norm(proportion_mean-row))
proportion_dists = np.array(proportion_dists)
proportion_dists.max()

In [ ]:
proportion_dists.argmax()

In [ ]:
df.iloc[115]['Body']